

Ссылка на регламент: https://docs.google.com/document/d/1YhUEX9fZDNTeE3eJ-yXskxZG46LsTRYvXjZ9Ij-t3Gw

In [ ]:
!pip install faiss-cpu langchain==0.0.271 openai==0.28.1 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import os
import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
import matplotlib.pyplot as plt
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
import tiktoken
from langchain.docstore.document import Document
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
data_from_url= load_document_text('https://docs.google.com/document/d/1YhUEX9fZDNTeE3eJ-yXskxZG46LsTRYvXjZ9Ij-t3Gw')

In [ ]:
data_from_url

'\ufeffТЕХНИЧЕСКИЙ РЕГЛАМЕНТ\r\nТАМОЖЕННОГО СОЮЗА "О БЕЗОПАСНОСТИ ЖЕЛЕЗНОДОРОЖНОГО\r\nПОДВИЖНОГО СОСТАВА"\r\n \r\n(ТР ТС 001/2011)\r\n \r\n\r\n\r\n\t \r\nI. Область применения\r\n \r\n1. Настоящий технический регламент распространяется на вновь разрабатываемые (модернизируемые), изготавливаемые железнодорожный подвижной состав с конструкционной скоростью до 200 км/ч включительно и его составные части (далее - продукция), выпускаемые в обращение на таможенной территории Евразийского экономического союза (далее - Союз) для использования на железнодорожных путях общего и необщего пользования с шириной колеи 1 520 мм.\r\nЖелезнодорожный подвижной состав включает в себя:\r\nлокомотивы;\r\nмоторвагонный подвижной состав и его вагоны;\r\nпассажирские вагоны локомотивной тяги;\r\nгрузовые вагоны;\r\nспециальный железнодорожный подвижной состав.\r\nПеречень объектов технического регулирования, на которые распространяются требования настоящего технического регламента, приведен в приложении N 1.\



---



CharacterTextSplitter

* separator - это символ или строка, используемые для разделения текста на чанки. Например, может использоваться символ новой строки ("\n").

* chunk_size - это максимальное количество символов, которые могут быть в каждом чанке.

* chunk_overlap - это количество символов, которые будут перекрываться между соседними чанками. Например, если указать значение 0, это означает, что нет перекрытия между чанками.

In [ ]:
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)

for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))

In [ ]:
len(source_chunks)

146

In [ ]:
# первый чанк
print(source_chunks[0])
page_content = source_chunks[0].page_content
# длина первого чанка
len(page_content)

page_content='\ufeffТЕХНИЧЕСКИЙ РЕГЛАМЕНТ\r\nТАМОЖЕННОГО СОЮЗА "О БЕЗОПАСНОСТИ ЖЕЛЕЗНОДОРОЖНОГО\r\nПОДВИЖНОГО СОСТАВА"\r\n \r\n(ТР ТС 001/2011)\r\n \r\n\r\n\r\n\t \r\nI. Область применения\r\n \r\n1. Настоящий технический регламент распространяется на вновь разрабатываемые (модернизируемые), изготавливаемые железнодорожный подвижной состав с конструкционной скоростью до 200 км/ч включительно и его составные части (далее - продукция), выпускаемые в обращение на таможенной территории Евразийского экономического союза (далее - Союз) для использования на железнодорожных путях общего и необщего пользования с шириной колеи 1 520 мм.\r\nЖелезнодорожный подвижной состав включает в себя:\r\nлокомотивы;\r\nмоторвагонный подвижной состав и его вагоны;\r\nпассажирские вагоны локомотивной тяги;\r\nгрузовые вагоны;\r\nспециальный железнодорожный подвижной состав.\r\nПеречень объектов технического регулирования, на которые распространяются требования настоящего технического регламента, приведен в при

1022



---



RecursiveCharacterTextSplitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))
len(source_chunks)

167

In [ ]:
source_chunks[0]

Document(page_content='\ufeffТЕХНИЧЕСКИЙ РЕГЛАМЕНТ\r\nТАМОЖЕННОГО СОЮЗА "О БЕЗОПАСНОСТИ ЖЕЛЕЗНОДОРОЖНОГО\r\nПОДВИЖНОГО СОСТАВА"\r\n \r\n(ТР ТС 001/2011)\r\n \r\n\r\n\r\n\t \r\nI. Область применения\r\n \r\n1. Настоящий технический регламент распространяется на вновь разрабатываемые (модернизируемые), изготавливаемые железнодорожный подвижной состав с конструкционной скоростью до 200 км/ч включительно и его составные части (далее - продукция), выпускаемые в обращение на таможенной территории Евразийского экономического союза (далее - Союз) для использования на железнодорожных путях общего и необщего пользования с шириной колеи 1 520 мм.\r\nЖелезнодорожный подвижной состав включает в себя:\r\nлокомотивы;\r\nмоторвагонный подвижной состав и его вагоны;\r\nпассажирские вагоны локомотивной тяги;\r\nгрузовые вагоны;\r\nспециальный железнодорожный подвижной состав.\r\nПеречень объектов технического регулирования, на которые распространяются требования настоящего технического регламента, приве



---


In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

similarity_search

In [ ]:
query = "На какие вагоны распространяется регламент?"
docs = db.similarity_search(query, k=4)

print("**отрезок №1**", docs[0].page_content)
print("**отрезок №2**", docs[1].page_content)
print("**отрезок №3**", docs[2].page_content)
print("**отрезок №4**", docs[3].page_content)

**отрезок №1** 2. Настоящий технический регламент не распространяется на вагоны санитарные, вагоны врачебных и диагностических поездов, тяговый железнодорожный подвижной состав, эксплуатируемый на железнодорожных путях необщего пользования без выхода на железнодорожные пути общего пользования, железнодорожный подвижной состав технологического железнодорожного транспорта организаций, предназначенный для перемещения людей и материальных ценностей на территории организаций и выполнения начально-конечных операций с железнодорожным подвижным составом для собственных нужд организаций.
3. Настоящий технический регламент устанавливает обязательные для применения и исполнения на таможенной территории Союза требования к продукции, применяемые при ее проектировании и производстве, правила идентификации продукции, требования к маркировке и правила ее нанесения, а также формы, схемы и процедуры оценки соответствия продукции.
**отрезок №2** ﻿ТЕХНИЧЕСКИЙ РЕГЛАМЕНТ
ТАМОЖЕННОГО СОЮЗА "О БЕЗОПАСНОСТИ ЖЕ

In [ ]:
query = "Какие части желензнодорожного состава нужно декларировать?"
docs = db.similarity_search(query, k=4)

print("**отрезок №1**", docs[0].page_content)
print("**отрезок №2**", docs[1].page_content)
print("**отрезок №3**", docs[2].page_content)
print("**отрезок №4**", docs[3].page_content)

**отрезок №1** перечень составных частей железнодорожного подвижного состава, подлежащих декларированию соответствия на основании собственных доказательств заявителя (при наличии) и доказательств, полученных с участием органа по сертификации и (или) аккредитованной испытательной лаборатории (центра), или декларированию соответствия на основании собственных доказательств заявителя (при наличии) и доказательств, полученных в том числе с участием органа по сертификации систем менеджмента и аккредитованной испытательной лаборатории (центра), согласно приложению N 4;
перечень составных частей железнодорожного подвижного состава, подлежащих декларированию соответствия на основании собственных доказательств заявителя, согласно приложению N 5;
перечень положений технического регламента Таможенного союза "О безопасности железнодорожного подвижного состава" (ТР ТС 001/2011), применяемых при сертификации железнодорожного подвижного состава, согласно приложению N 6;
**отрезок №2** 97. В эксплуатац

similarity_search_with_score

In [ ]:
docs_and_scores = db.similarity_search_with_score(query, k=4)

In [ ]:
print("Контент: ", docs_and_scores[0][0].page_content)

Контент:  перечень составных частей железнодорожного подвижного состава, подлежащих декларированию соответствия на основании собственных доказательств заявителя (при наличии) и доказательств, полученных с участием органа по сертификации и (или) аккредитованной испытательной лаборатории (центра), или декларированию соответствия на основании собственных доказательств заявителя (при наличии) и доказательств, полученных в том числе с участием органа по сертификации систем менеджмента и аккредитованной испытательной лаборатории (центра), согласно приложению N 4;
перечень составных частей железнодорожного подвижного состава, подлежащих декларированию соответствия на основании собственных доказательств заявителя, согласно приложению N 5;
перечень положений технического регламента Таможенного союза "О безопасности железнодорожного подвижного состава" (ТР ТС 001/2011), применяемых при сертификации железнодорожного подвижного состава, согласно приложению N 6;


In [ ]:
print("Оценка расстояния: ", docs_and_scores[0][1])

Оценка расстояния:  0.25893024


In [ ]:
# второй фрагмент
print("Контент: ", docs_and_scores[1][0].page_content)
print("Оценка расстояния: ", docs_and_scores[1][1])

Контент:  14. При проектировании железнодорожного подвижного состава и его составных частей проектировщик (разработчик) принимает решения, обеспечивающие установленный законодательством государств-членов допустимый уровень вредного и (или) опасного воздействия на жизнь и здоровье человека, животных и растений.
15. Выбранные проектировщиком (разработчиком) конструкции железнодорожного подвижного состава и его составных частей должны быть безопасны в течение назначенного срока службы и (или) до достижения назначенного ресурса, в течение назначенного срока хранения, а также выдерживать воздействия и нагрузки, которым они могут подвергаться в процессе эксплуатации.
16. При проектировании железнодорожного подвижного состава и его составных частей проектировщик (разработчик) должен предусматривать системы для защиты обслуживающего персонала и (или) пассажиров в случае столкновения и (или) схода железнодорожного подвижного состава с рельсов.
Оценка расстояния:  0.31127232


In [ ]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

In [ ]:
system="Ты-сотрудник таможенного союза, ответь на вопрос клиента на основе документа с информацией (ТЕХНИЧЕСКИЙ РЕГЛАМЕНТ ТАМОЖЕННОГО СОЮЗА О БЕЗОПАСНОСТИ ЖЕЛЕЗНОДОРОЖНОГО ПОДВИЖНОГО СОСТАВА). Не придумывай ничего от себя, отвечай максимально по документу. Не упоминай Документ с информацией для ответа. Юзер ничего не должен знать про Документ с информацией для ответа юзеру"

In [ ]:
topic= "На какие вагоны распространяется регламент?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================2. Настоящий технический регламент не распространяется на вагоны санитарные, вагоны врачебных и диагностических поездов, тяговый железнодорожный подвижной состав, эксплуатируемый на железнодорожных путях необщего пользования без выхода на железнодорожные пути общего пользования, железнодорожный подвижной состав технологического железнодорожного транспорта организаций, предназначенный для перемещения людей и материальных ценностей на территории организаций и выполнения начально-конечных операций с железнодорожным подвижным составом для собственных нужд организаций.
3. Настоящий технический регламент устанавливает обязательные для применения и исполнения на таможенной территории Союза требования к продукции, применяемые при ее проектировании и производстве, правила идентификации продукции, требования к маркировке и правила ее нанесения, а также формы, схемы и процедуры оценки соо

' Регламент о безопасности железнодорожного подвижного состава распространяется на различные типы вагонов, включая вагоны бункерного типа, изотермические вагоны, крытые  вагоны, пассажирские вагоны локомотивной тяги, вагоны-платформы, вагоны-самосвалы, вагоны-цистерны, вагоны широкой колеи для промышленности, дизель-поезда, автомотрисы,  рельсовые автобусы, полувагоны, специальный несамоходный и самоходный железнодорожный подвижной состав, тепловозы, газотурбовозы, транспортеры железнодорожные,  электровозы магистральные и маневровые, а также скоростные пассажирские вагоны и моторвагонный подвижной состав.'

In [ ]:
topic= "Какие части желензнодорожного состава нужно декларировать?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================перечень составных частей железнодорожного подвижного состава, подлежащих декларированию соответствия на основании собственных доказательств заявителя (при наличии) и доказательств, полученных с участием органа по сертификации и (или) аккредитованной испытательной лаборатории (центра), или декларированию соответствия на основании собственных доказательств заявителя (при наличии) и доказательств, полученных в том числе с участием органа по сертификации систем менеджмента и аккредитованной испытательной лаборатории (центра), согласно приложению N 4;
перечень составных частей железнодорожного подвижного состава, подлежащих декларированию соответствия на основании собственных доказательств заявителя, согласно приложению N 5;
перечень положений технического регламента Таможенного союза "О безопасности железнодорожного подвижного состава" (ТР ТС 001/2011), применяемых при сертификаци

' Для декларирования соответствия железнодорожного подвижного состава необходимо указывать составные части, которые подлежат декларированию на основании собственных  доказательств заявителя и доказательств, полученных с участием органа по сертификации и (или) аккредитованной испытательной лаборатории (центра), а также на основании  собственных доказательств заявителя и доказательств, полученных в том числе с участием органа по сертификации систем менеджмента и аккредитованной испытательной  лаборатории (центра).'